# 1 对poem.xlsx中的诗歌进行向前最大匹配分词，词表在wordlist.xlsx中已经给出

注意：有些字符可能不在词表中（out-of-vocabulary），可以保留这些字符，也可以直接去掉

In [9]:
# 读取文件，如有报错，可根据报错信息安装xlrd或者openpyxl
import pandas as pd
import numpy as np
table = pd.read_excel('poem_v2.xlsx')
word_list_df = pd.read_excel('wordlist_v2.xlsx')


In [11]:
word_list_df = word_list_df[word_list_df.simple !='一作']

In [3]:
#存储所有分词
wl = word_list_df['simple'].tolist()

In [5]:
# 计算wordlist中词语的最大长度max_length
word_list_df['len'] = word_list_df['simple'].apply(len)
max_length = word_list_df.sort_values('len', ascending = False).iloc[0, 2]
max_length

3

In [6]:
#分词的过程

# 实现前向最大匹配分词算法
# string: 待分词的串
# word_list: 词表
# max_length: 词表中词语的最大长度
import string
def cut(stri, word_list, max_length):
    stri = ''.join(c for c in stri if c not in string.punctuation).replace(' ','')
    result = []
    cur = stri[:max_length]
    while len(stri):
        for i in range(max_length+1)[::-1]:
            if cur[:i] in word_list:
                result.append(cur[:i])
                stri = stri[i:]
                cur = stri[:max_length]
                break
            elif i == 1:#如果某个字不在list中直接去除
                stri = stri[i:]
                cur = stri[:max_length]
                break
                
    return result
# 执行分词（可能需要等待几分钟）
table['words'] = table['simple'].apply(lambda x: ' '.join(cut(x, wl, max_length)))
table.to_csv('wordlist_v3.csv', encoding = 'utf_8_sig', index = False)

In [49]:
#直接读取分词结果
table = pd.read_csv('wordlist_v3.csv')
table1 = table.drop(['content', 'words'], axis = 1)

In [51]:
#根据诗歌id输出诗歌得内容
def gather(x):
    res = []
    res.append(x[x.line_number==-100]['simple'].values[0][2:])
    res.append(x[x.line_number==-1]['simple'].values[0][2:])
    res.extend(list(x[(x.line_number!=-100)&(x.line_number!=-1)]['simple'].values))
    return res
id_poems = pd.DataFrame(table.groupby('Poem_id').apply(gather))

In [ ]:
#根据作者的名字，找出对应的诗歌id
author = table1[table1.line_number == -1]
author['simple'] = author['simple'].apply(lambda x:x[2:])
author_poem_list = author.groupby('simple').apply(lambda x:x['Poem_id'].values)

# 2 统计每个词在文档中的TF-IDF值

注意：本次作业实现最基础版本的TF-IDF计算即可，不必实现其他变种

In [ ]:
# 按照空格分开，stack
split_words = table['words'].str.split(' ', expand=True).stack().rename('word').reset_index()
new_data = pd.merge(table['Poem_id'], split_words, left_index=True, right_on='level_0')
new_data

In [ ]:
df = new_data.loc[:,['word', 'Poem_id']]
df_s = df.sort_values('word')
df_s1 = df_s[df_s.word != '']
#计算tf值
df_s1['freq'] = 1
df_s2 = pd.DataFrame(df_s1.groupby(['word','Poem_id'])['freq'].sum())

In [ ]:
#展成向量形式
word_vec = df_s2.unstack()
word_vec.columns = word_vec.columns.droplevel()

In [ ]:
#计算idf
import numpy as np
df = word_vec.count(axis = 1)
N = word_vec.shape[1]
idf = np.log(N/df)

In [18]:
#tf-idf矩阵
#weight = word_vec.apply(lambda x:x*idf)
weight = pd.read_csv('weight1.csv', index_col = 'word', engine = 'c', encoding = "utf-8")

In [ ]:
weight

In [ ]:
#根据分词索引存储过它的诗歌id
'''
def search1(query):
    l = weight.loc[query]
    doc = l[l==l].sort_values(ascending = False)
    res = doc.index.values
    return list(res)

df = pd.DataFrame({'word':weight.index.values})
def search(x):
    return search1(x.word)
df['list'] = df.apply(search, axis = 1)
df = df.set_index('word')
'''
#df.to_csv('word_id.csv', encoding = 'utf-8_sig')
dataf = pd.read_csv('word_id.csv')

In [ ]:
dataf['doc_freq'] = dataf.apply(lambda x:len(x.list), axis =1)
dataf['word_freq'] = df_s2.groupby('word')['freq'].count()

In [ ]:
all_words = weight.index.values
all_words

In [60]:
split_words = table['words'].str.split(' ', expand=True).stack().rename('word').reset_index()
new_data = pd.merge(table['Poem_id'], split_words, left_index=True, right_on='level_0')
new_data

,Poem_id,level_0,level_1,word
0,4371,0,0,饯
1,4371,0,1,唐
2,4371,0,2,永昌
3,4371,0,3,一
4,4371,0,4,作
...,...,...,...,...
200373,39205,46275,4,屏
200374,39205,46276,0,尽日
200375,39205,46276,1,池边
200376,39205,46276,2,钓


# 3.查询算法

In [15]:
#近义词表
similar_dict = np.load('similar_dict.npy',allow_pickle=True).item()

In [61]:
new_data['freq']=1
a=new_data.groupby('word')['freq'].sum()
target_list = a[a>10].index.values
target_list

array(['䌽', '一', '一一', ..., '龙', '龙门', '龟'], dtype=object)

In [151]:
pd.DataFrame(target_list).to_csv('target_list.csv',encoding = 'utf_8_sig')

In [31]:
MAX_POEM_SHOWN = 20
#查询关键词只有一个
def search1(weight, query,similar_dict):
    l = weight.loc[query]
    doc = l[l==l].sort_values(ascending = False)
    res = list(doc.index.values)
    num = len(res)
    print(num)
    #如果符合条件的诗歌太少，进行近义词搜索
    if len(res) < MAX_POEM_SHOWN:
        similar_d = similar_dict[query]
        i = 0
        while num < MAX_POEM_SHOWN:
            cur_key = similar_d[i][0]
            print(cur_key)
            l = weight.loc[cur_key]
            doc = l[l==l].sort_values(ascending = False)
            res.extend(list(doc.index.values))
            i+=1
            num += len(doc)
    return res[:min(20,len(res))]

In [145]:
#查询关键词有两个
def search2(weight, query_list,similar_dict):    
    word1 = query_list[0]
    word2 = query_list[1]
    l1 = weight.loc[word1]
    doc1 = pd.DataFrame(l1[l1==l1]).reset_index()
    l2 = weight.loc[word2]
    doc2 = pd.DataFrame(l2[l2==l2]).reset_index()

    #取交集
    alpha = 0.6
    intersection = pd.merge(doc1, doc2, on = 'index', how = 'inner')
    intersection['weight'] = intersection[word1] * alpha + intersection[word2] * (2-alpha)
    intersection = intersection.loc[:,['index', 'weight']]
    concat_list = []
    if len(intersection)>0:
        concat_list.append(intersection)

    if len(intersection) < MAX_POEM_SHOWN:
        if word1 in target_list:
            similar1 = similar_dict[word1][0][0]
            similar1_cor = similar_dict[word1][0][1]
            l1 = weight.loc[similar1]
            doc11 = pd.DataFrame(l1[l1==l1]).reset_index()
            intersection1 = pd.merge(doc11, doc2, on = 'index', how = 'inner')
            intersection1['weight'] = intersection1[similar1] * similar1_cor * alpha + intersection1[word2] * (2-alpha)
            intersection1 = intersection1.loc[:,['index', 'weight']]
            if len(intersection)>0:
                concat_list.append(intersection1)

        if word2 in target_list:
            similar2 = similar_dict[word2][0][0]
            similar2_cor = similar_dict[word2][0][1]
            l2 = weight.loc[similar2]
            doc21 = pd.DataFrame(l2[l2==l2]).reset_index()
            intersection2 = pd.merge(doc1, doc21, on = 'index', how = 'inner')
            intersection2['weight'] = intersection2[word1] *  alpha + intersection2[similar2] * similar2_cor *(2-alpha)
            intersection2 = intersection2.loc[:,['index', 'weight']]
            if len(intersection)>0:
                concat_list.append(intersection2)


    union = pd.merge(doc1, doc2, on = 'index', how = 'outer')
    left = union.append(intersection).drop_duplicates(keep = False)
    left['weight'] = left.apply(lambda x: x.iloc[1] if x.iloc[1] == x.iloc[1] else x.iloc[2], axis = 1)
    left = left.loc[:,['index','weight']]
    concat_list.append(left)
    concat = pd.concat(concat_list).drop_duplicates('index')
    concat = concat.sort_values('weight', ascending = False)
    return concat['index'].values[:min(len(concat),20)]

In [146]:
#查询关键词有三个
def search3(weight, query,similar_dict):
    word1 = query_list[0]
    word2 = query_list[1]
    word3 = query_list[2]
    l1 = weight.loc[word1]
    doc1 = pd.DataFrame(l1[l1==l1]).reset_index()
    l2 = weight.loc[word2]
    doc2 = pd.DataFrame(l2[l2==l2]).reset_index()
    l3 = weight.loc[word3]
    doc3 = pd.DataFrame(l3[l3==l3]).reset_index()

    #取三个关键词的交集
    intersection1 = pd.merge(doc1, doc2, on = 'index', how = 'inner')
    intersection2 = pd.merge(doc1, doc3, on = 'index', how = 'inner')
    intersection3 = pd.merge(doc2, doc3, on = 'index', how = 'inner')

    intersection = pd.merge(intersection1, doc3, on = 'index', how = 'inner')
    concat_list = []
    alpha = 1.1
    beta=1
    intersection['weight'] = intersection[word1] * alpha + intersection[word2] * beta + intersection[word3] * (3-alpha-beta)
    concat_list = []
    intersection=intersection.loc[:,['index','weight']]
    concat_list.append(intersection)
    
    #只包含两个关键词的集合
    left1 = intersection1.append(intersection).drop_duplicates(keep = False)
    left1['weight'] = left1.apply(lambda x: alpha*x.iloc[1]+(2-alpha)*x.iloc[2] if x.weight != x.weight else x.iloc[3], axis = 1)
    left1 = left1.loc[:,['index','weight']]

    left2 = intersection2.append(intersection).drop_duplicates(keep = False)
    left2['weight'] = left2.apply(lambda x: alpha*x.iloc[1]+(2-alpha)*x.iloc[2] if x.weight != x.weight else x.iloc[3], axis = 1)
    left2 = left2.loc[:,['index','weight']]

    left3 = intersection3.append(intersection).drop_duplicates(keep = False)
    left3['weight'] = left3.apply(lambda x: alpha*x.iloc[1]+(2-alpha)*x.iloc[2] if x.weight != x.weight else x.iloc[3], axis = 1)
    left3 = left3.loc[:,['index','weight']]
    
    #关键词替换：只替换一个
    if len(intersection) < MAX_POEM_SHOWN:
        if len(intersection1) >0 and word3 in target_list:
            similar3 = similar_dict[word3][0][0]
            similar3_cor = similar_dict[word3][0][1]
            l31 = weight.loc[similar3]
            doc31 = pd.DataFrame(l31[l31==l31]).reset_index()
            intersection31 = pd.merge(intersection1, doc31, on = 'index', how = 'inner')
            intersection31['weight'] = intersection31[word1] * alpha+intersection31[word2]* beta +intersection31[similar3]* (3-alpha-beta)*similar3_cor
            intersection31=intersection31.loc[:,['index','weight']]
            concat_list.append(intersection31)
        if len(intersection2) >0 and word2 in target_list:
            similar2 = similar_dict[word2][0][0]
            similar2_cor = similar_dict[word2][0][1]
            l21 = weight.loc[similar2]
            doc21 = pd.DataFrame(l21[l21==l21]).reset_index()
            intersection21 = pd.merge(intersection2, doc21, on = 'index', how = 'inner')
            intersection21['weight'] = intersection21[word1]*alpha+intersection21[similar2]*beta*similar2_cor +intersection21[word3]*(3-alpha-beta)
            intersection21=intersection21.loc[:,['index','weight']]
            concat_list.append(intersection21)
        if len(intersection3) >0 and word1 in target_list:
            similar1 = similar_dict[word1][0][0]
            similar1_cor = similar_dict[word1][0][1]
            l11 = weight.loc[similar1]
            doc11 = pd.DataFrame(l11[l11==l11]).reset_index()
            intersection11 = pd.merge(intersection3, doc11, on = 'index', how = 'inner')
            intersection11['weight'] = intersection11[similar1]*alpha*similar1_cor+intersection11[word2]* beta +intersection11[word3]* (3-alpha-beta)
            intersection11=intersection11.loc[:,['index','weight']]
            concat_list.append(intersection11)
    concat_list.extend([left1,left2,left3])
    res = pd.concat(concat_list).sort_values('weight', ascending = False)
    return res['index'].values[:min(len(res),20)]

In [147]:
#输入关键词字符串
def query_word(weight, query_list,similar_dict):  
    if len(query_list) == 1:
        ret = search1(weight, query_list[0],similar_dict)
    elif len(query_list) == 2:
        ret = search2(weight, query_list,similar_dict)
    elif len(query_list) == 3:
        ret = search3(weight, query_list,similar_dict)
    else:
        ret = 'too many words'
    return ret

In [148]:
query_word(weight,['夜', '明月', '一'])

array(['18933', '34102', '29603', '34102', '10950', '29603', '34102',
       '10950', '10950', '10950', '29603', '34102', '29603', '29127',
       '22530', '22312', '30089', '18294', '20385', '29603'], dtype=object)